# A prototype of a loan demand prediction model

* 서울시 신한은행의 대출 수요를 예측합니다. 
* 사용데이터 
    1. 신한은행 제공 - 서울시 지역단위 '소득', '지출', '금융자산' 정보
    2. 서울시 공공데이터 포털 - 서울시 주민등록인구 (동별) 통계 (2019년 1분기/3분기, 2020년 1분기/3분기)
    3. 서울시 공공데이터 포털 - 주택 매매가격 지수, 주택 전세 가격 지수 
    4. 은행연합회 소비자 포털 - 대출금리 비교 
    [시중은행 대출금리 비교](https://portal.kfb.or.kr/compare/loan_household.php)

# 데이터 호출 및 분석준비  

In [1]:
import pandas as pd 

In [2]:
sinhan = pd.read_excel('../finance_data201903(1)/total.xlsx')

In [3]:
interest = pd.read_excel('C:/Users/wnsgn/Desktop/금융분석기타데이터/대출금리비교(일반신용대출).xlsx')
population = pd.read_excel('C:/Users/wnsgn/Desktop/금융분석기타데이터/동별인구최종.xlsx')
sell_price = pd.read_excel('C:/Users/wnsgn/Desktop/금융분석기타데이터/매매가격지수.xlsx')
rent_price = pd.read_excel('C:/Users/wnsgn/Desktop/금융분석기타데이터/전세가격지수.xlsx')
dong = pd.read_excel('C:/Users/wnsgn/Desktop/금융분석기타데이터/서울시_법정동_행정동.xlsx')

In [5]:
# dong = pd.read_excel('C:/Users/wnsgn/Desktop/금융분석기타데이터/서울시_법정동_행정동.xlsx')

In [6]:
sinhan

,기준년월,지역구,법정동코드,법정동,집계구코드,나이,성별,직장인여부,급여입금,가맹점매출입금,연금입금,총소비금액,총수신금액,예적금금액,신탁금액,수익증권금액,신용대출금액,담보대출금액,주택대출금액,전세자금대출금액
0,201903,강남구,11680101,역삼동,1123064010001,2,1,0,0,0,0,2018319,2430312,0,0,0,0,0,0,0
1,201903,강남구,11680101,역삼동,1123064010001,2,2,0,0,0,0,2326837,2221251,0,0,0,0,0,0,0
2,201903,강남구,11680101,역삼동,1123064010001,3,1,0,0,0,0,1771639,1382057,1841250,0,0,0,0,0,0
3,201903,강남구,11680101,역삼동,1123064010001,3,2,0,0,0,0,1745601,2902578,3561440,0,0,0,0,0,0
4,201903,강남구,11680101,역삼동,1123064010001,4,1,0,0,0,0,1712433,1746992,8831666,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039563,202103,중랑구,11260106,신내동,1107070010802,5,2,0,0,0,0,1425069,4369530,9185200,0,0,0,0,0,0
1039564,202103,중랑구,11260106,신내동,1107070010802,6,1,0,0,0,738450,2015000,6877740,2478000,0,0,0,0,0,0
1039565,202103,중랑구,11260106,신내동,1107070010802,6,2,0,0,0,595480,1495569,9297919,19789456,0,0,0,0,0,0
1039566,202103,중랑구,11260106,신내동,1107070010802,7,1,0,0,0,409938,812073,5018284,0,0,0,0,0,0,0


In [7]:
loan = sinhan.groupby(['기준년월','지역구','법정동'])['담보대출금액','주택대출금액','전세자금대출금액'].sum()
loan = loan.reset_index()
loan

<ipython-input-7-0699ed4fb877>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  loan = sinhan.groupby(['기준년월','지역구','법정동'])['담보대출금액','주택대출금액','전세자금대출금액'].sum()


,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액
0,201903,강남구,개포동,6485038182,715940254,753808290
1,201903,강남구,논현동,2646678768,0,0
2,201903,강남구,대치동,9850664028,682082615,0
3,201903,강남구,도곡동,7463640865,273515488,0
4,201903,강남구,삼성동,5094494299,456449547,0
...,...,...,...,...,...,...
2273,202103,중랑구,면목동,2900528317,790272336,585895405
2274,202103,중랑구,묵동,1019469554,0,102685714
2275,202103,중랑구,상봉동,2967801116,0,1045166859
2276,202103,중랑구,신내동,1411840760,78397019,38832607


In [8]:
loan['대출총액'] = loan['담보대출금액'] + loan['주택대출금액'] + loan["전세자금대출금액"]

In [9]:
loan

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액
0,201903,강남구,개포동,6485038182,715940254,753808290,7954786726
1,201903,강남구,논현동,2646678768,0,0,2646678768
2,201903,강남구,대치동,9850664028,682082615,0,10532746643
3,201903,강남구,도곡동,7463640865,273515488,0,7737156353
4,201903,강남구,삼성동,5094494299,456449547,0,5550943846
...,...,...,...,...,...,...,...
2273,202103,중랑구,면목동,2900528317,790272336,585895405,4276696058
2274,202103,중랑구,묵동,1019469554,0,102685714,1122155268
2275,202103,중랑구,상봉동,2967801116,0,1045166859,4012967975
2276,202103,중랑구,신내동,1411840760,78397019,38832607,1529070386


In [10]:
# 이자율 데이터 
interest

,기준년월,기준금리,업계평균 대출금리,신한은행 대출금리
0,2019-03-01,1.75,4.759412,3.83
1,2019-09-01,1.50,4.088235,3.08
2,2020-03-01,1.25,3.958235,3.11
3,2020-09-01,0.50,3.295882,2.39
4,2021-03-01,0.50,3.935294,3.09


In [11]:
loan['기준년월'] = pd.to_datetime(loan['기준년월'], format='%Y%m')
loan

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액
0,2019-03-01,강남구,개포동,6485038182,715940254,753808290,7954786726
1,2019-03-01,강남구,논현동,2646678768,0,0,2646678768
2,2019-03-01,강남구,대치동,9850664028,682082615,0,10532746643
3,2019-03-01,강남구,도곡동,7463640865,273515488,0,7737156353
4,2019-03-01,강남구,삼성동,5094494299,456449547,0,5550943846
...,...,...,...,...,...,...,...
2273,2021-03-01,중랑구,면목동,2900528317,790272336,585895405,4276696058
2274,2021-03-01,중랑구,묵동,1019469554,0,102685714,1122155268
2275,2021-03-01,중랑구,상봉동,2967801116,0,1045166859,4012967975
2276,2021-03-01,중랑구,신내동,1411840760,78397019,38832607,1529070386


In [12]:
data_set = pd.merge(loan, interest, left_on='기준년월', right_on='기준년월', how = 'inner')
data_set

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액,기준금리,업계평균 대출금리,신한은행 대출금리
0,2019-03-01,강남구,개포동,6485038182,715940254,753808290,7954786726,1.75,4.759412,3.83
1,2019-03-01,강남구,논현동,2646678768,0,0,2646678768,1.75,4.759412,3.83
2,2019-03-01,강남구,대치동,9850664028,682082615,0,10532746643,1.75,4.759412,3.83
3,2019-03-01,강남구,도곡동,7463640865,273515488,0,7737156353,1.75,4.759412,3.83
4,2019-03-01,강남구,삼성동,5094494299,456449547,0,5550943846,1.75,4.759412,3.83
...,...,...,...,...,...,...,...,...,...,...
2273,2021-03-01,중랑구,면목동,2900528317,790272336,585895405,4276696058,0.50,3.935294,3.09
2274,2021-03-01,중랑구,묵동,1019469554,0,102685714,1122155268,0.50,3.935294,3.09
2275,2021-03-01,중랑구,상봉동,2967801116,0,1045166859,4012967975,0.50,3.935294,3.09
2276,2021-03-01,중랑구,신내동,1411840760,78397019,38832607,1529070386,0.50,3.935294,3.09


In [13]:
dong

,행정동코드,시도명,시군구명,행정동,법정동코드,법정동,생성일자
0,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101
1,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101
2,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101
3,1111051500,서울특별시,종로구,청운효자동,1111010400,효자동,20081101
4,1111051500,서울특별시,종로구,청운효자동,1111010500,창성동,20081101
...,...,...,...,...,...,...,...
739,1174065000,서울특별시,강동구,성내제2동,1174010800,성내동,19880423
740,1174066000,서울특별시,강동구,성내제3동,1174010800,성내동,19880423
741,1174068500,서울특별시,강동구,길동,1174010500,길동,20080707
742,1174069000,서울특별시,강동구,둔촌제1동,1174010600,둔촌동,19880423


In [14]:
행정동_to_법정동 = dong[['행정동', '법정동']]
행정동_to_법정동 

,행정동,법정동
0,청운효자동,청운동
1,청운효자동,신교동
2,청운효자동,궁정동
3,청운효자동,효자동
4,청운효자동,창성동
...,...,...
739,성내제2동,성내동
740,성내제3동,성내동
741,길동,길동
742,둔촌제1동,둔촌동


In [15]:
population

,기간,자치구,행정동,세대,인구
0,2019.1/4,종로구,사직동,4552,9782
1,2019.1/4,종로구,삼청동,1400,3045
2,2019.1/4,종로구,부암동,4295,10523
3,2019.1/4,종로구,평창동,7552,19002
4,2019.1/4,종로구,무악동,3111,8585
...,...,...,...,...,...
2118,2021.1/4,강동구,둔촌제1동,81,146
2119,2021.1/4,강동구,둔촌제2동,10648,25309
2120,2021.1/4,강동구,암사제1동,15932,35245
2121,2021.1/4,강동구,천호제2동,17480,34526


In [16]:
population = pd.merge(population, 행정동_to_법정동, left_on='행정동', right_on='행정동', how='left')

In [17]:
population.loc[population['행정동']=='상일동','법정동']= '상일동'

In [18]:
population.loc[population['행정동']=='상일동']

,기간,자치구,행정동,세대,인구,법정동
727,2019.1/4,강동구,상일동,4185,11127,상일동
1471,2019.3/4,강동구,상일동,4154,10943,상일동
2216,2020.1/4,강동구,상일동,9586,27636,상일동
2961,2020.1/4,강동구,상일동,9586,27636,상일동
3706,2021.1/4,강동구,상일동,13090,37989,상일동


In [19]:
pd.to_datetime('201903',format='%Y%m')

Timestamp('2019-03-01 00:00:00')

In [20]:
population.loc[population['기간']=='2019.1/4', '기간'] = pd.to_datetime('201903',format='%Y%m')
population.loc[population['기간']=='2019.3/4', '기간'] = pd.to_datetime('201909',format='%Y%m')
population.loc[population['기간']=='2020.1/4', '기간'] = pd.to_datetime('202003',format='%Y%m')
population.loc[population['기간']=='2020.3/4', '기간'] = pd.to_datetime('202009',format='%Y%m')
population.loc[population['기간']=='2021.1/4', '기간'] = pd.to_datetime('202103',format='%Y%m')

In [21]:
population

,기간,자치구,행정동,세대,인구,법정동
0,2019-03-01 00:00:00,종로구,사직동,4552,9782,통의동
1,2019-03-01 00:00:00,종로구,사직동,4552,9782,적선동
2,2019-03-01 00:00:00,종로구,사직동,4552,9782,체부동
3,2019-03-01 00:00:00,종로구,사직동,4552,9782,필운동
4,2019-03-01 00:00:00,종로구,사직동,4552,9782,내자동
...,...,...,...,...,...,...
3718,2021-03-01 00:00:00,강동구,둔촌제1동,81,146,둔촌동
3719,2021-03-01 00:00:00,강동구,둔촌제2동,10648,25309,둔촌동
3720,2021-03-01 00:00:00,강동구,암사제1동,15932,35245,암사동
3721,2021-03-01 00:00:00,강동구,천호제2동,17480,34526,천호동


In [22]:
population.rename(columns={'기간':'기준년월'}, inplace=True)

In [23]:
# population = population.groupby(by=['기준년월','법정동'])['세대','인구'].sum().reset_index()

In [24]:
population = population.groupby(by=['법정동'])['세대','인구'].mean().reset_index()

<ipython-input-24-b73328fb12af>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  population = population.groupby(by=['법정동'])['세대','인구'].mean().reset_index()


In [25]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2277
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   기준년월       2278 non-null   datetime64[ns]
 1   지역구        2278 non-null   object        
 2   법정동        2278 non-null   object        
 3   담보대출금액     2278 non-null   int64         
 4   주택대출금액     2278 non-null   int64         
 5   전세자금대출금액   2278 non-null   int64         
 6   대출총액       2278 non-null   int64         
 7   기준금리       2278 non-null   float64       
 8   업계평균 대출금리  2278 non-null   float64       
 9   신한은행 대출금리  2278 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(2)
memory usage: 195.8+ KB


In [26]:
population

,법정동,세대,인구
0,가락동,11166.666667,29207.933333
1,가리봉동,5873.600000,15103.000000
2,가산동,14327.600000,24892.400000
3,가양동,12700.900000,26841.200000
4,가회동,2052.800000,4444.800000
...,...,...,...
460,후암동,8564.000000,18367.600000
461,훈정동,5244.600000,8535.000000
462,휘경동,9441.600000,20451.800000
463,흑석동,14224.600000,35036.800000


In [27]:
population.loc[population['법정동']=='항동']

,법정동,세대,인구
438,항동,5697.666667,15060.0


In [28]:
# temp_data1 = data_set.loc[data_set["기준년월"]== pd.to_datetime('201903',format='%Y%m')] 
# temp_pop1 = population.loc[population['기준년월']== pd.to_datetime('201903',format='%Y%m')]
# temp1 = pd.merge(left=temp_data1,right=temp_pop1[["법정동", "세대","인구"]], left_on='법정동',right_on='법정동', how='left')

# temp_data2 = data_set.loc[data_set["기준년월"]== pd.to_datetime('201909',format='%Y%m')] 
# temp_pop2 = population.loc[population['기준년월']== pd.to_datetime('201909',format='%Y%m')]
# temp2 = pd.merge(left=temp_data2,right=temp_pop2[["법정동", "세대","인구"]], left_on='법정동',right_on='법정동', how='left')

# temp_data3 = data_set.loc[data_set["기준년월"]== pd.to_datetime('202003',format='%Y%m')] 
# temp_pop3 = population.loc[population['기준년월']== pd.to_datetime('202003',format='%Y%m')]
# temp3 = pd.merge(left=temp_data3,right=temp_pop3[["법정동", "세대","인구"]], left_on='법정동',right_on='법정동', how='left')

# temp_data4 = data_set.loc[data_set["기준년월"]== pd.to_datetime('202009',format='%Y%m')] 
# temp_pop4 = population.loc[population['기준년월']== pd.to_datetime('202009',format='%Y%m')]
# temp4 = pd.merge(left=temp_data4,right=temp_pop4[["법정동", "세대","인구"]], left_on='법정동',right_on='법정동', how='left')

# temp_data5 = data_set.loc[data_set["기준년월"]== pd.to_datetime('202103',format='%Y%m')] 
# temp_pop5 = population.loc[population['기준년월']== pd.to_datetime('202103',format='%Y%m')]
# temp5 = pd.merge(left=temp_data5,right=temp_pop5[["법정동", "세대","인구"]], left_on='법정동',right_on='법정동', how='left')

# data_set = temp1.append(temp2)
# data_set = data_set.append(temp3)
# data_set = data_set.append(temp4)
# data_set = data_set.append(temp5)

# data_set.reset_index(drop=True,inplace=True)

In [29]:
data_set

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액,기준금리,업계평균 대출금리,신한은행 대출금리
0,2019-03-01,강남구,개포동,6485038182,715940254,753808290,7954786726,1.75,4.759412,3.83
1,2019-03-01,강남구,논현동,2646678768,0,0,2646678768,1.75,4.759412,3.83
2,2019-03-01,강남구,대치동,9850664028,682082615,0,10532746643,1.75,4.759412,3.83
3,2019-03-01,강남구,도곡동,7463640865,273515488,0,7737156353,1.75,4.759412,3.83
4,2019-03-01,강남구,삼성동,5094494299,456449547,0,5550943846,1.75,4.759412,3.83
...,...,...,...,...,...,...,...,...,...,...
2273,2021-03-01,중랑구,면목동,2900528317,790272336,585895405,4276696058,0.50,3.935294,3.09
2274,2021-03-01,중랑구,묵동,1019469554,0,102685714,1122155268,0.50,3.935294,3.09
2275,2021-03-01,중랑구,상봉동,2967801116,0,1045166859,4012967975,0.50,3.935294,3.09
2276,2021-03-01,중랑구,신내동,1411840760,78397019,38832607,1529070386,0.50,3.935294,3.09


In [30]:
data_set = pd.merge(left=data_set,right=population[["법정동", "세대","인구"]], left_on='법정동',right_on='법정동', how='left')

In [31]:
data_set

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액,기준금리,업계평균 대출금리,신한은행 대출금리,세대,인구
0,2019-03-01,강남구,개포동,6485038182,715940254,753808290,7954786726,1.75,4.759412,3.83,6414.250000,16259.300000
1,2019-03-01,강남구,논현동,2646678768,0,0,2646678768,1.75,4.759412,3.83,12868.800000,22892.300000
2,2019-03-01,강남구,대치동,9850664028,682082615,0,10532746643,1.75,4.759412,3.83,9468.800000,25616.000000
3,2019-03-01,강남구,도곡동,7463640865,273515488,0,7737156353,1.75,4.759412,3.83,10211.400000,28452.600000
4,2019-03-01,강남구,삼성동,5094494299,456449547,0,5550943846,1.75,4.759412,3.83,9627.300000,22560.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2021-03-01,중랑구,면목동,2900528317,790272336,585895405,4276696058,0.50,3.935294,3.09,11166.266667,23592.433333
2274,2021-03-01,중랑구,묵동,1019469554,0,102685714,1122155268,0.50,3.935294,3.09,11964.500000,27906.300000
2275,2021-03-01,중랑구,상봉동,2967801116,0,1045166859,4012967975,0.50,3.935294,3.09,10715.500000,22615.600000
2276,2021-03-01,중랑구,신내동,1411840760,78397019,38832607,1529070386,0.50,3.935294,3.09,11900.600000,31139.400000


In [32]:
data_set.loc[data_set['인구'].isnull()]

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액,기준금리,업계평균 대출금리,신한은행 대출금리,세대,인구


In [33]:
sell_price

,기간,자치구,종합,아파트
0,201711,기준,100.0,100.0
1,201903,서울시,106.2,107.7
2,201903,종로구,106.2,108.4
3,201903,중구,108.1,110.3
4,201903,용산구,109.4,110.1
...,...,...,...,...
568,202012,관악구,107.9,110.9
569,202012,서초구,110.7,111.8
570,202012,강남구,112.1,113.9
571,202012,송파구,112.8,118.2


In [34]:
sell_price.rename(columns={'기간':'기준년월','자치구':'지역구' ,'종합':'매매종합지수'}, inplace=True)
sell_price

,기준년월,지역구,매매종합지수,아파트
0,201711,기준,100.0,100.0
1,201903,서울시,106.2,107.7
2,201903,종로구,106.2,108.4
3,201903,중구,108.1,110.3
4,201903,용산구,109.4,110.1
...,...,...,...,...
568,202012,관악구,107.9,110.9
569,202012,서초구,110.7,111.8
570,202012,강남구,112.1,113.9
571,202012,송파구,112.8,118.2


In [35]:
rent_price.rename(columns={'기간':'기준년월','자치구':'지역구' , '종합':'전세종합지수'}, inplace=True)

In [36]:
rent_price

,기준년월,지역구,전세종합지수,아파트
0,201711,기준,100.0,100.0
1,201903,서울시,99.3,98.4
2,201903,종로구,102.8,103.1
3,201903,중구,101.4,101.9
4,201903,용산구,99.6,98.4
...,...,...,...,...
568,202012,관악구,103.3,106.6
569,202012,서초구,104.2,105.3
570,202012,강남구,103.9,105.4
571,202012,송파구,104.3,106.1


In [37]:
sell_price['기준년월'] = pd.to_datetime(sell_price['기준년월'], format='%Y%m')
rent_price['기준년월'] = pd.to_datetime(rent_price['기준년월'], format="%Y%m")

In [38]:
sell_price

,기준년월,지역구,매매종합지수,아파트
0,2017-11-01,기준,100.0,100.0
1,2019-03-01,서울시,106.2,107.7
2,2019-03-01,종로구,106.2,108.4
3,2019-03-01,중구,108.1,110.3
4,2019-03-01,용산구,109.4,110.1
...,...,...,...,...
568,2020-12-01,관악구,107.9,110.9
569,2020-12-01,서초구,110.7,111.8
570,2020-12-01,강남구,112.1,113.9
571,2020-12-01,송파구,112.8,118.2


In [39]:
rent_price

,기준년월,지역구,전세종합지수,아파트
0,2017-11-01,기준,100.0,100.0
1,2019-03-01,서울시,99.3,98.4
2,2019-03-01,종로구,102.8,103.1
3,2019-03-01,중구,101.4,101.9
4,2019-03-01,용산구,99.6,98.4
...,...,...,...,...
568,2020-12-01,관악구,103.3,106.6
569,2020-12-01,서초구,104.2,105.3
570,2020-12-01,강남구,103.9,105.4
571,2020-12-01,송파구,104.3,106.1


In [40]:
data_set.loc[data_set['기준년월']!=pd.to_datetime("202103", format='%Y%m')]

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액,기준금리,업계평균 대출금리,신한은행 대출금리,세대,인구
0,2019-03-01,강남구,개포동,6485038182,715940254,753808290,7954786726,1.75,4.759412,3.83,6414.250000,16259.300000
1,2019-03-01,강남구,논현동,2646678768,0,0,2646678768,1.75,4.759412,3.83,12868.800000,22892.300000
2,2019-03-01,강남구,대치동,9850664028,682082615,0,10532746643,1.75,4.759412,3.83,9468.800000,25616.000000
3,2019-03-01,강남구,도곡동,7463640865,273515488,0,7737156353,1.75,4.759412,3.83,10211.400000,28452.600000
4,2019-03-01,강남구,삼성동,5094494299,456449547,0,5550943846,1.75,4.759412,3.83,9627.300000,22560.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
1816,2020-09-01,중랑구,면목동,1712802427,272112750,111442857,2096358034,0.50,3.295882,2.39,11166.266667,23592.433333
1817,2020-09-01,중랑구,묵동,704738613,0,0,704738613,0.50,3.295882,2.39,11964.500000,27906.300000
1818,2020-09-01,중랑구,상봉동,1740739705,0,544253450,2284993155,0.50,3.295882,2.39,10715.500000,22615.600000
1819,2020-09-01,중랑구,신내동,1551316984,155495803,38599440,1745412227,0.50,3.295882,2.39,11900.600000,31139.400000


In [41]:
temp_sell1 = sell_price.loc[sell_price['기준년월'] == pd.to_datetime('201903', format='%Y%m')]
temp_rent1 = rent_price.loc[rent_price['기준년월'] == pd.to_datetime('201903', format='%Y%m')]
temp_data1 = data_set.loc[data_set['기준년월'] == pd.to_datetime('201903', format='%Y%m')]
temp1 = pd.merge(left=temp_data1, right=temp_sell1[["지역구", "매매종합지수"]], left_on='지역구', right_on='지역구', how='left')
temp1 = pd.merge(left=temp1, right=temp_rent1[["지역구", "전세종합지수"]], left_on='지역구', right_on='지역구', how='left')

temp_sell2 = sell_price.loc[sell_price['기준년월'] == pd.to_datetime('201909', format='%Y%m')]
temp_rent2 = rent_price.loc[rent_price['기준년월'] == pd.to_datetime('201909', format='%Y%m')]
temp_data2 = data_set.loc[data_set['기준년월'] == pd.to_datetime('201909', format='%Y%m')]
temp2 = pd.merge(left=temp_data2, right=temp_sell2[["지역구", "매매종합지수"]], left_on='지역구', right_on='지역구', how='left')
temp2 = pd.merge(left=temp2, right=temp_rent2[["지역구", "전세종합지수"]], left_on='지역구', right_on='지역구', how='left')

temp_sell3 = sell_price.loc[sell_price['기준년월'] == pd.to_datetime('202003', format='%Y%m')]
temp_rent3 = rent_price.loc[rent_price['기준년월'] == pd.to_datetime('202003', format='%Y%m')]
temp_data3 = data_set.loc[data_set['기준년월'] == pd.to_datetime('202003', format='%Y%m')]
temp3 = pd.merge(left=temp_data3, right=temp_sell3[["지역구", "매매종합지수"]], left_on='지역구', right_on='지역구', how='left')
temp3 = pd.merge(left=temp3, right=temp_rent3[["지역구", "전세종합지수"]], left_on='지역구', right_on='지역구', how='left')

temp_sell4 = sell_price.loc[sell_price['기준년월'] == pd.to_datetime('202009', format='%Y%m')]
temp_rent4 = rent_price.loc[rent_price['기준년월'] == pd.to_datetime('202009', format='%Y%m')]
temp_data4 = data_set.loc[data_set['기준년월'] == pd.to_datetime('202009', format='%Y%m')]
temp4 = pd.merge(left=temp_data4, right=temp_sell4[["지역구", "매매종합지수"]], left_on='지역구', right_on='지역구', how='left')
temp4 = pd.merge(left=temp4, right=temp_rent4[["지역구", "전세종합지수"]], left_on='지역구', right_on='지역구', how='left')


data_set = temp1.append(temp2)
data_set = data_set.append(temp3)
data_set = data_set.append(temp4)
data_set.reset_index(drop=True,inplace=True)
data_set

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액,기준금리,업계평균 대출금리,신한은행 대출금리,세대,인구,매매종합지수,전세종합지수
0,2019-03-01,강남구,개포동,6485038182,715940254,753808290,7954786726,1.75,4.759412,3.83,6414.250000,16259.300000,106.7,95.2
1,2019-03-01,강남구,논현동,2646678768,0,0,2646678768,1.75,4.759412,3.83,12868.800000,22892.300000,106.7,95.2
2,2019-03-01,강남구,대치동,9850664028,682082615,0,10532746643,1.75,4.759412,3.83,9468.800000,25616.000000,106.7,95.2
3,2019-03-01,강남구,도곡동,7463640865,273515488,0,7737156353,1.75,4.759412,3.83,10211.400000,28452.600000,106.7,95.2
4,2019-03-01,강남구,삼성동,5094494299,456449547,0,5550943846,1.75,4.759412,3.83,9627.300000,22560.600000,106.7,95.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,2020-09-01,중랑구,면목동,1712802427,272112750,111442857,2096358034,0.50,3.295882,2.39,11166.266667,23592.433333,107.8,101.7
1817,2020-09-01,중랑구,묵동,704738613,0,0,704738613,0.50,3.295882,2.39,11964.500000,27906.300000,107.8,101.7
1818,2020-09-01,중랑구,상봉동,1740739705,0,544253450,2284993155,0.50,3.295882,2.39,10715.500000,22615.600000,107.8,101.7
1819,2020-09-01,중랑구,신내동,1551316984,155495803,38599440,1745412227,0.50,3.295882,2.39,11900.600000,31139.400000,107.8,101.7


In [42]:
data_set.loc[data_set['인구'].isnull()]

,기준년월,지역구,법정동,담보대출금액,주택대출금액,전세자금대출금액,대출총액,기준금리,업계평균 대출금리,신한은행 대출금리,세대,인구,매매종합지수,전세종합지수
